In [1]:
import pandas as pd

In [31]:
bruk_sents = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/bruk_parallel.csv")
ng_sents = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/ng_parallel.csv")

In [6]:
bruk_sents.head()

,original_sentence,orig_sent_id,changed_sentence,changed_sent_id,original_file_name,orig_ann,changed_ann
0,Його редактором був поет-символіст Яків Савчен...,46,Його редакторкою була поетеса-символістка Анже...,46,e5e76a8efa0f.txt,"{'T30': ('поет-символіст', 'JOB')}","{'T30': ('поетеса-символістка', 'JOB')}"
1,Міжпредметні паралелі . Маніфест футуристів ск...,62,Міжпредметні паралелі . Маніфест футуристок ск...,62,e5e76a8efa0f.txt,"{'T49': ('поет', 'JOB')}","{'T49': ('поетеса', 'JOB')}"
2,Помітною була організація « Гарт » ( 1923 — 19...,96,Помітною була організація « Гарт » ( 1923 — 19...,96,e5e76a8efa0f.txt,"{'T104': ('поет', 'JOB')}","{'T104': ('поетка', 'JOB')}"
3,Її очолив байкар і прозаїк Сергій Пилипенко .,127,Її очолила байкарка і прозаїкиня Марія Пилипен...,127,e5e76a8efa0f.txt,"{'T143': ('байкар', 'JOB'), 'T144': ('прозаїк'...","{'T143': ('байкарка', 'JOB'), 'T144': ('прозаї..."
4,У Галицько-Волинському літописі згадується спі...,171,У Галицько-Волинському літописі згадується спі...,171,e5e76a8efa0f.txt,"{'T218': ('співець', 'JOB')}","{'T218': ('співчиня', 'JOB')}"


In [38]:
all_sentences = pd.concat([bruk_sents, ng_sents])

In [7]:
test_swapped = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/utils_files/test_swapped.csv", index_col=0)
dev_swapped = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/utils_files/dev_swapped.csv", index_col=0)
df_after_annotation = pd.concat([dev_swapped, test_swapped]).reset_index()

In [20]:
doubled_sents = []

for x, row in df_after_annotation.iterrows():
    if row['Оригінальне речення'] == row['Змінене речення'] and row['Коректність речення'] == 'Правильне':
        doubled_sents.append(row['Оригінальне речення'])

In [23]:
dismiss_sents = list(df_after_annotation[df_after_annotation['Коректність речення']=='Важко визначити']['Оригінальне речення'].values)

In [17]:
len(dismiss_sents)

59

In [25]:
bad_sentes_need_to_be_removed = dismiss_sents + doubled_sents

In [26]:
len(bad_sentes_need_to_be_removed)

119

## Filter par sentences

In [40]:
all_sentences.shape

(1513, 7)

In [42]:
all_sentences_filtered = all_sentences[
    (~all_sentences['original_sentence'].isin(doubled_sents)) &
    (~all_sentences['original_sentence'].isin(dismiss_sents))
]


In [43]:
all_sentences_filtered.shape

(1403, 7)

In [45]:
all_sentences_filtered.to_csv("all_parallel_sentence_both_datasets_with_filenames_and_labels.csv", index=0)

## Find files with bad sentences

In [ ]:
bruk_sent_c, ng_sents_c = {}, {}

for bad_sent in bad_sentes_need_to_be_removed:

    if bad_sent in ng_sents['original_sentence'].values:

        filename = ng_sents[ng_sents['original_sentence']==bad_sent]['original_file_name'].iloc[0]

        if filename in ng_sents_c.keys():
            ng_sents_c[filename].append(bad_sent)
        else:
            ng_sents_c[filename] = [bad_sent]

    elif bad_sent in bruk_sents['original_sentence'].values:

        filename = bruk_sents[bruk_sents['original_sentence']==bad_sent]['original_file_name'].iloc[0]

        if filename in bruk_sent_c.keys():
            bruk_sent_c[filename].append(bad_sent)
        else:
            bruk_sent_c[filename] = [bad_sent]

In [ ]:
import os

def update_ann_for_removed_sentences(txt_path, ann_path, sentences_to_remove):
    with open(txt_path, "r", encoding="utf-8") as f:
        original_text = f.read()

    sentences = original_text.split('\n')
    sentence_spans = []
    start = 0
    for s in sentences:
        end = start + len(s)
        sentence_spans.append((start, end))
        start = end + 1

    sentences_to_remove_set = set(sentences_to_remove)
    removed_spans = []
    shift_mapping = {}
    new_sentences = []
    current_index = 0

    for s, (start, end) in zip(sentences, sentence_spans):
        if s in sentences_to_remove_set:
            removed_spans.append((start, end))
            continue
        new_sentences.append(s)
        shift_mapping[(start, end)] = (current_index, current_index + len(s))
        current_index += len(s) + 1

    new_text = '\n'.join(new_sentences)

    updated_ann_lines = []
    new_tid_counter = 1

    with open(ann_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.startswith("T"):
                updated_ann_lines.append(line.strip())
                continue

            parts = line.strip().split('\t')
            if len(parts) != 5:
                continue

            tid, label, start_str, end_str, entity = parts
            start, end = int(start_str), int(end_str)

            if any(rs <= start < re or rs < end <= re or (start <= rs and end >= re) for (rs, re) in removed_spans):
                continue

            for (orig_start, orig_end), (new_start, _) in shift_mapping.items():
                if orig_start <= start < orig_end:
                    offset = new_start - orig_start
                    new_ann_start = start + offset
                    new_ann_end = end + offset
                    new_tid = f"T{new_tid_counter}"
                    new_line = f"{new_tid}\t{label}\t{new_ann_start}\t{new_ann_end}\t{entity}"
                    updated_ann_lines.append(new_line)
                    new_tid_counter += 1
                    break

    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(new_text)
    with open(ann_path, "w", encoding="utf-8") as f:
        f.write('\n'.join(updated_ann_lines) + '\n')

    print(f"✅ Updated files saved to:\n- {txt_path}\n- {ann_path}")


In [61]:
def get_deleted_sentence_lengths(file_path, delete_list):
    sentence_lengths = []

    with open(file_path, 'r', encoding='utf-8') as f:
        sentences = f.read().split('\n')

    for sentence in sentences:
        if sentence in delete_list:
            print(len(sentence) + 1, sentence)

    return sentence_lengths